In [1]:
import keras
import numpy as np
import pandas as pd
import cv2
import csv

Using TensorFlow backend.
/root/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
from keras.models import Sequential, Model,load_model
from keras.layers import Dense, Activation, Flatten, Dropout, Conv2D, MaxPooling2D, concatenate, Input, Lambda, Cropping2D
from keras import utils
from keras.callbacks import Callback, LambdaCallback, EarlyStopping, ModelCheckpoint, TensorBoard
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras import backend as K
from keras.preprocessing import image
from keras.layers.normalization import BatchNormalization

In [3]:
def logger(epoch, logs):
    if epoch %2== 0: 
        print(epoch, logs)
logging_callback = LambdaCallback(
    on_epoch_end=logger)

In [5]:
# data2 is my own training images
lines = []
with open('data_sample/driving_log.csv') as f:
    reader = csv.reader(f)
    for line in reader:
        lineout = [x.replace('/Users/wwymak/simulator_files', '.') for x in line]
        lines.append(lineout)

In [5]:
images = []
stearing = []
for line in lines[1:]:
    img_path = line[0]
    image = cv2.imread('data_sample/' + img_path)
    images.append(image)
    stearing.append(line[3]) 

In [6]:
X_train = np.array(images)
y_train = np.array(stearing)

In [11]:
X_train_norm = (X_train / 255.) - 0.5

In [28]:
model = Sequential()
Lambda(lambda x: (x / 255.0) - 0.5)
model.add(Flatten(input_shape=X_train[0].shape))
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, shuffle=True, epochs=6,validation_split=0.2, callbacks=[ stopping_callback])

Train on 6428 samples, validate on 1608 samples
Epoch 1/6
6428/6428 [==============================] - 4s 652us/step - loss: 5085274.4461 - val_loss: 3896.2098

In [29]:
model.save('model1_simple.h5')

In [8]:
def model2(ksize=(3,3), dropout=0.25):
    inputs = Input(shape=X_train[0].shape)
    processed = Lambda(lambda x: (x /255. - 0.5))(inputs)
    block1 = Conv2D(32, kernel_size=ksize, activation='relu', padding='same', name='set1_conv1')(processed)
    block1 = BatchNormalization()(block1)
    block1 = Conv2D(32, kernel_size=ksize, activation='relu', padding='same', name='set1_conv2')(block1)
    block1 = BatchNormalization()(block1)
    block1 = MaxPooling2D(pool_size=(2, 2),strides=(2, 2), name='set1_pool')(block1)
    
    block2 = Conv2D(64, kernel_size=ksize, activation='relu', padding='same', name='set2_conv1')(block1)
    block2 = BatchNormalization()(block2)
    block2 = Conv2D(64, kernel_size=ksize, activation='relu', padding='same', name='set2_conv2')(block2)
    block2 = BatchNormalization()(block2)
    block2 = Conv2D(64, kernel_size=ksize, activation='relu', padding='same', name='set2_conv3')(block2)
    block2 = BatchNormalization()(block2)
    block2 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2),name='set2_pool')(block2)
    output2 = Flatten()(block2)
        
    fcblock = Dense(1024, activation='relu', name='fc1')(output2)
    fcblock = BatchNormalization()(fcblock)
    fcblock = Dropout(dropout)(fcblock)
    fcblock = Dense(512, activation='relu' , name='fc2')(fcblock)
    fcblock = BatchNormalization()(fcblock)
    fcblock = Dropout(dropout)(fcblock)
    
    predictions = Dense(1, name='final')(fcblock)
    model = Model(inputs=inputs, outputs=predictions)
    model.compile(optimizer='adam',
              loss='mse')
    
    return model

In [9]:
K.clear_session()
batch_size = 8
epochs = 5
model = model2(ksize=(5,5))

history = model.fit(X_train, y_train, batch_size=batch_size,
                    epochs=epochs,validation_split=0.2,
                      verbose=1,shuffle=True, 
                    callbacks=[logging_callback, ModelCheckpoint('./models/vgg_carsim1.h5', save_best_only=True)]
                   )

Train on 6428 samples, validate on 1608 samples
Epoch 1/5
6428/6428 [==============================] - 436s 68ms/step - loss: 0.7635 - val_loss: 0.0442
Epoch 2/5
6428/6428 [==============================] - 427s 66ms/step - loss: 0.0863 - val_loss: 0.0232

In [11]:
keras.__version__

'2.1.1'

In [5]:
def leNet(ksize=(3,3),dropout=0.25):
    inputs = Input(shape=X_train[0].shape)
#     processed = Lambda(lambda x: (x /255. - 0.5))(inputs)
    
    block1 = Conv2D(32, kernel_size=ksize, activation='relu', padding='same', name='set1_conv1')(inputs)
    block1 = BatchNormalization()(block1)
    block1 = MaxPooling2D(pool_size=(2, 2),strides=(2, 2), name='set1_pool')(block1)
    
    block2 = Conv2D(32, kernel_size=ksize, activation='relu', padding='same', name='set2_conv1')(block1)
    block2 = BatchNormalization()(block2)
    block2 = MaxPooling2D(pool_size=(2, 2),strides=(2, 2), name='set2_pool')(block2)
    
    output2 = Flatten()(block2)
    fcblock = Dense(128, activation='relu', name='fc1')(output2)
    fcblock = BatchNormalization()(fcblock)
    fcblock = Dropout(dropout)(fcblock)
    fcblock = Dense(84, activation='relu' , name='fc2')(fcblock)
    fcblock = BatchNormalization()(fcblock)
    fcblock = Dropout(dropout)(fcblock)
    
    predictions = Dense(1, name='final')(fcblock)
    model = Model(inputs=inputs, outputs=predictions)
    model.compile(optimizer='adam',
              loss='mse')
    
    return model

In [15]:
K.clear_session()
batch_size = 32
epochs = 5
lenet_model2 = leNet(ksize=(5,5))

lehistory2 = lenet_model2.fit(X_train, y_train, batch_size=batch_size,
                    epochs=epochs,validation_split=0.2,
                      verbose=1,shuffle=True, 
                    callbacks=[logging_callback, ModelCheckpoint('./models/lenet_carsim2.h5', save_best_only=True)]
                   )

Train on 1764 samples, validate on 441 samples
Epoch 1/5
1764/1764 [==============================] - 16s 9ms/step - loss: 1.3489 - val_loss: 54.5161
Epoch 2/5
1764/1764 [==============================] - 14s 8ms/step - loss: 0.8405 - val_loss: 29.0661

In [17]:
lehistory2 = lenet_model2.fit(X_train, y_train, batch_size=batch_size,
                    epochs=10,validation_split=0.2,
                      verbose=1,shuffle=True, 
                    callbacks=[logging_callback, ModelCheckpoint('./models/lenet_carsim2.h5', save_best_only=True)]
                   )

Train on 1764 samples, validate on 441 samples
Epoch 1/10
1764/1764 [==============================] - 14s 8ms/step - loss: 0.3600 - val_loss: 0.1150
Epoch 2/10
1764/1764 [==============================] - 13s 8ms/step - loss: 0.3039 - val_loss: 0.1059

### Data Augmentation

In [6]:
lines = []
with open('data_sample/driving_log.csv') as f:
    reader = csv.reader(f)
    for line in reader:
        lineout = [x.replace('/Users/wwymak/simulator_files', '') for x in line]
        lines.append(lineout)

images = []
stearing = []
for line in lines[1:]:
    img_path = line[0]
    image = cv2.imread('data_sample/' + img_path)
    images.append(image)
    stearing.append(line[3]) 
augmented_img = []
augmented_measurements = []

for img, measurement in zip(images, stearing):
    augmented_img.append(img)
    augmented_measurements.append(float(measurement))
    augmented_img.append(np.fliplr(img))
    augmented_measurements.append(float(measurement) * -1.0)

In [7]:
X_train = np.array(augmented_img)
y_train = np.array(augmented_measurements)

In [29]:
K.clear_session()
batch_size = 32
epochs = 15
lenet_model = leNet(ksize=(5,5))

lehistory = lenet_model.fit(X_train, y_train, batch_size=batch_size,
                    epochs=epochs,validation_split=0.2,
                      verbose=1,shuffle=True, 
                    callbacks=[logging_callback, ModelCheckpoint('./models/lenet_carsim_aug.h5', save_best_only=True)]
                   )

Train on 3928 samples, validate on 982 samples
Epoch 1/15
3928/3928 [==============================] - 29s 7ms/step - loss: 1.1093 - val_loss: 0.3885
Epoch 2/15
3928/3928 [==============================] - 29s 7ms/step - loss: 0.5076 - val_loss: 0.1541

In [6]:
# use all 3 cameras:
images = []
steering = []

augmented_img = []
augmented_measurements = []

for line in lines[1:]:
    center_img_path = line[0]
    left_img_path = line[1]
    right_img_path = line[2]
    steering_angle = float(line[3])
    
    image_left = cv2.imread('data2/' + left_img_path)
    image_center = cv2.imread('data2/' + center_img_path)
    image_right = cv2.imread('data2/' + right_img_path)
    
    images.append(image_center)
    steering.append(steering_angle) 
    images.append(image_left)
    steering.append(steering_angle + 0.01 * steering_angle) 
    images.append(image_right)
    steering.append(steering_angle - 0.01 * steering_angle) 

for img, measurement in zip(images, steering):
    augmented_img.append(img)
    augmented_measurements.append(float(measurement))
    augmented_img.append(np.fliplr(img))
    augmented_measurements.append(float(measurement) * -1.0)

In [7]:
X_train = np.array(augmented_img)
y_train = np.array(augmented_measurements)

In [14]:
X_train.shape

(13230, 160, 320, 3)

In [8]:
K.clear_session()
batch_size = 32
epochs = 15
lenet_model = leNet(ksize=(5,5))

lehistory_extra_cameras = lenet_model.fit(X_train, y_train, batch_size=batch_size,
                    epochs=epochs,validation_split=0.2,
                      verbose=0,shuffle=True, 
                    callbacks=[logging_callback, ModelCheckpoint('./models/lenet_carsim_extra_cameras_aug.h5', save_best_only=True)]
                   )

0 {'val_loss': 12.902390845784479, 'loss': 0.5866002030248274}
2 {'val_loss': 0.08850899513301215, 'loss': 0.10809890164494605}
4 {'val_loss': 0.092939633865174093, 'loss': 0.070776603952647163}
6 {'val_loss': 0.098443242767016187, 'loss': 0.056847289376360667}
8 {'val_loss': 0.10994071064577442, 'loss': 0.044426237746160856}
10 {'val_loss': 0.1290239747754458, 'loss': 0.031522306819880719}
12 {'val_loss': 0.10665588607646959, 'loss': 0.022608343982056786}
14 {'val_loss': 0.1053542845149666, 'loss': 0.018948854753242147}


In [20]:
def nvidia_model():
    inputs = Input(shape=X_train[0].shape)
    cropped = Cropping2D(cropping=((70, 25), (0, 0)))(inputs)
    processed = Lambda(lambda x: (x /255. - 0.5))(cropped) 
    
    block1 = Conv2D(24, kernel_size=5,strides=(2,2), activation='relu', padding='same', name='set1_conv1')(processed)
    block1 = BatchNormalization()(block1)
    block2 = Conv2D(36, kernel_size=5,strides=(2,2), activation='relu', padding='same', name='set2_conv1')(block1)
    block2 = BatchNormalization()(block2)
    block3 = Conv2D(48, kernel_size=5,strides=(2,2), activation='relu', padding='same', name='set3_conv1')(block2)
    block3 = BatchNormalization()(block3)
    block4 = Conv2D(64, kernel_size=3, activation='relu', padding='same', name='set4_conv1')(block3)
    block4 = BatchNormalization()(block4)
    block5 = Conv2D(64, kernel_size=3, activation='relu', padding='same', name='set5_conv1')(block4)
    block5 = BatchNormalization()(block5)
    
    flat1 = Flatten()(block5)
    
    fcblock = Dense(100, activation='relu', name='fc1')(flat1)
    fcblock = Dense(50, activation='relu', name='fc2')(fcblock)
    fcblock = Dense(10, activation='relu' , name='fc3')(fcblock)
    
    predictions = Dense(1, name='final')(fcblock)
    model = Model(inputs=inputs, outputs=predictions)
    model.compile(optimizer='adam',
              loss='mse')
    
    return model

In [1]:
def nvidia_model_no_lambda():
    inputs = Input(shape=X_train[0].shape)
    cropped = Cropping2D(cropping=((70, 25), (0, 0)))(inputs)
#     processed = Lambda(lambda x: (x /255. - 0.5))(cropped) 
    
    block1 = Conv2D(24, kernel_size=5,strides=(2,2), activation='relu', padding='same', name='set1_conv1')(cropped)
    block2 = Conv2D(36, kernel_size=5,strides=(2,2), activation='relu', padding='same', name='set2_conv1')(block1)
    block3 = Conv2D(48, kernel_size=5,strides=(2,2), activation='relu', padding='same', name='set3_conv1')(block2)
    block4 = Conv2D(64, kernel_size=3, activation='relu', padding='same', name='set4_conv1')(block3)
    block5 = Conv2D(64, kernel_size=3, activation='relu', padding='same', name='set5_conv1')(block4)
    
    flat1 = Flatten()(block5)
    
    fcblock = Dense(100, activation='relu', name='fc1')(flat1)
    fcblock = Dense(50, activation='relu', name='fc2')(fcblock)
    fcblock = Dense(10, activation='relu' , name='fc3')(fcblock)
    
    predictions = Dense(1, name='final')(fcblock)
    model = Model(inputs=inputs, outputs=predictions)
    model.compile(optimizer='adam',
              loss='mse')
    
    return model

In [10]:
K.clear_session()
batch_size = 32
epochs = 10
nvidia_model = nvidia_model()

nvidia_model1_hist = nvidia_model.fit(X_train, y_train, batch_size=batch_size,
                    epochs=epochs,validation_split=0.2,
                      verbose=1,shuffle=True, 
                    callbacks=[logging_callback, ModelCheckpoint('./models/nvidia_model1.h5', save_best_only=True)]
                   )

Train on 3928 samples, validate on 982 samples
Epoch 1/10
3928/3928 [==============================] - 28s 7ms/step - loss: 0.0598 - val_loss: 0.0504
Epoch 2/10
3928/3928 [==============================] - 6s 2ms/step - loss: 0.0555 - val_loss: 0.0504

In [ ]:
# using the nvidia model on the sample data, with reversing, and alos using 3 cameras + extra recovery images
driving_logs = ['data_sample/driving_log.csv' ]
with open('data_sample/driving_log.csv') as f:
    reader = csv.reader(f)
    for line in reader:
        lineout = [x.replace('/Users/wwymak/simulator_files', '.').replace(' ', '') for x in line]
        lines.append(lineout)

In [4]:
data_file_paths = ['data2/', 'data_recovery/', 'data_reverse/']
images = []
steering = []

In [5]:
# normal center cam + augmentation
lines = []
with open(data_file_paths[0] + '/driving_log.csv') as f:
    reader = csv.reader(f)
    for line in reader:
        lineout = [x.replace('/Users/wwymak/simulator_files', '').replace(' ', '') for x in line]
        lines.append(lineout)

images_set1 = []
steering_set1 = []
for line in lines[1:]:
    img_path = line[0]
    image = cv2.imread(data_file_paths[0] + img_path)
    images_set1.append(image)
    steering_set1.append(line[3]) 

augmented_img = []
augmented_measurements = []

for img, measurement in zip(images_set1, steering_set1):
    augmented_img.append(img)
    augmented_measurements.append(float(measurement))
    augmented_img.append(np.fliplr(img))
    augmented_measurements.append(float(measurement) * -1.0)


images += augmented_img
steering += augmented_measurements


In [6]:
images_set2 = []
steering_set2 = []

for line in lines[1:]:
    left_img_path = line[1]
    right_img_path = line[2]
    steering_angle = float(line[3])
    
    image_left = cv2.imread(data_file_paths[0] + left_img_path)
    image_right = cv2.imread(data_file_paths[0] + right_img_path)
    
    
    images_set2.append(image_left)
    steering_set2.append(steering_angle + 0.01 * steering_angle) 
    images_set2.append(image_right)
    steering_set2.append(steering_angle - 0.01 * steering_angle) 



In [7]:
images += images_set2
steering += steering_set2


In [9]:
# add in recovery images

lines = []
with open(data_file_paths[1] + '/driving_log.csv') as f:
    reader = csv.reader(f)
    for line in reader:
        lineout = [x.replace('/Users/wwymak/simulator_files2', '').replace(' ', '') for x in line]
        lines.append(lineout)

images_set3 = []
steering_set3 = []
for line in lines[1:]:
    img_path = line[0]
    image = cv2.imread(data_file_paths[1] + img_path)
    images_set3.append(image)
    steering_set3.append(line[3]) 




In [27]:
# images += images_set3
# steering += steering_set3


In [10]:
X_train = np.array(images)
y_train = np.array(steering)

In [11]:
X_train.shape

(8820, 160, 320, 3)

In [14]:
K.clear_session()
batch_size = 32
epochs = 15
nvidia_model = nvidia_model()

nvidia_model2_hist = nvidia_model.fit(X_train, y_train, batch_size=batch_size,
                    epochs=epochs,validation_split=0.2,
                      verbose=0,shuffle=True, 
                    callbacks=[logging_callback, ModelCheckpoint('./models/nvidia_model2_with_recovery.h5', save_best_only=True),
                              TensorBoard(log_dir='./logs' )]
                   )

0 {'val_loss': 0.1302447020923064, 'loss': 0.1391302009288416}
2 {'val_loss': 0.11151453729321897, 'loss': 0.086976764795461209}
4 {'val_loss': 0.11266344180436226, 'loss': 0.07536905605236148}
6 {'val_loss': 0.11543113973980047, 'loss': 0.06586703713856587}
8 {'val_loss': 0.10893718951409197, 'loss': 0.059045307397369358}
10 {'val_loss': 0.10292021720300591, 'loss': 0.049461517195785398}
12 {'val_loss': 0.10850077439226261, 'loss': 0.073800792279832758}
14 {'val_loss': 0.094179654371441088, 'loss': 0.054174397246333474}


In [13]:
K.clear_session()
batch_size = 32
epochs = 10
model_no_lambda= nvidia_model_no_lambda()

model_no_lambda_hist = model_no_lambda.fit(X_train, y_train, batch_size=batch_size,
                    epochs=epochs,validation_split=0.2,
                      verbose=0,shuffle=True, 
                    callbacks=[logging_callback, ModelCheckpoint('./models/nvidia_model_no_lambda.h5', save_best_only=True),
                              TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=32, )]
                   )

NameError: name 'nvidia_model_no_lambda' is not defined

In [4]:
data_file_paths = ['data_sample/', 'data_recovery/', 'data_reverse/']
images = []
steering = []
# normal center cam + augmentation
lines = []
with open(data_file_paths[0] + '/driving_log.csv') as f:
    reader = csv.reader(f)
    for line in reader:
        lineout = [x.replace('/Users/wwymak/simulator_files', '').replace(' ', '') for x in line]
        lines.append(lineout)

images_set1 = []
steering_set1 = []
for line in lines[1:]:
    img_path = line[0]
    image = cv2.imread(data_file_paths[0] + img_path)
    images_set1.append(image)
    steering_set1.append(line[3]) 

augmented_img = []
augmented_measurements = []

for img, measurement in zip(images_set1, steering_set1):
    augmented_img.append(img)
    augmented_measurements.append(float(measurement))
    augmented_img.append(np.fliplr(img))
    augmented_measurements.append(float(measurement) * -1.0)


images += augmented_img
steering += augmented_measurements


images_set2 = []
steering_set2 = []

for line in lines[1:]:
    left_img_path = line[1]
    right_img_path = line[2]
    steering_angle = float(line[3])
    
    image_left = cv2.imread(data_file_paths[0] + left_img_path)
    image_right = cv2.imread(data_file_paths[0] + right_img_path)
    
    
    images_set2.append(image_left)
    steering_set2.append(steering_angle + 0.008 * steering_angle) 
    images_set2.append(image_right)
    steering_set2.append(steering_angle - 0.008 * steering_angle) 

images += images_set2
steering += steering_set2

lines = []
with open(data_file_paths[1] + '/driving_log.csv') as f:
    reader = csv.reader(f)
    for line in reader:
        lineout = [x.replace('/Users/wwymak/simulator_files2', '').replace(' ', '') for x in line]
        lines.append(lineout)

images_set3 = []
steering_set3 = []
for line in lines[1:]:
    img_path = line[0]
    image = cv2.imread(data_file_paths[1] + img_path)
    images_set3.append(image)
    steering_set3.append(line[3]) 

images += images_set3
steering += steering_set3


X_train = np.array(images)
y_train = np.array(steering)

In [21]:
K.clear_session()
batch_size = 32
epochs = 15
nvidia = nvidia_model()

nvidia_model3_hist = nvidia.fit(X_train, y_train, batch_size=batch_size,
                    epochs=epochs,validation_split=0.2,
                      verbose=0,shuffle=True, 
                    callbacks=[logging_callback, ModelCheckpoint('./models/nvidia_model_sampledata_with_recovery.h5', save_best_only=True),
                              TensorBoard(log_dir='./logs' )]
                   )

0 {'val_loss': 0.064017859886459624, 'loss': 0.040343593935500917}
2 {'val_loss': 0.077520490694979099, 'loss': 0.010158982894465112}
4 {'val_loss': 0.13056416264322218, 'loss': 0.0093345073743430831}
6 {'val_loss': 0.058935736559686459, 'loss': 0.0087316353197393701}
8 {'val_loss': 0.054345160650844895, 'loss': 0.010095160219629267}
10 {'val_loss': 0.05810211673898899, 'loss': 0.0094746242202805257}
12 {'val_loss': 0.06125576982210839, 'loss': 0.01114470164173709}
14 {'val_loss': 0.055912538036727959, 'loss': 0.010371572792380365}


In [19]:
X_train_data2, y_train_data2

(35463,)

In [ ]:
#  with random shifts in steering angle
data_file_paths = ['data_sample/', 'data_recovery/', 'data_reverse/']
images = []
steering = []
# normal center cam + augmentation
lines = []
with open(data_file_paths[0] + '/driving_log.csv') as f:
    reader = csv.reader(f)
    for line in reader:
        lineout = [x.replace('/Users/wwymak/simulator_files', '').replace(' ', '') for x in line]
        lines.append(lineout)

images_set1 = []
steering_set1 = []
for line in lines[1:]:
    img_path = line[0]
    image = cv2.imread(data_file_paths[0] + img_path)
    images_set1.append(image)
    steering_set1.append(line[3]) 

augmented_img = []
augmented_measurements = []

for img, measurement in zip(images_set1, steering_set1):
    augmented_img.append(img)
    augmented_measurements.append(float(measurement))
    augmented_img.append(np.fliplr(img))
    augmented_measurements.append(float(measurement) * -1.0)


images += augmented_img
steering += augmented_measurements


images_set2 = []
steering_set2 = []

for line in lines[1:]:
    left_img_path = line[1]
    right_img_path = line[2]
    steering_angle = float(line[3])
    
    image_left = cv2.imread(data_file_paths[0] + left_img_path)
    image_right = cv2.imread(data_file_paths[0] + right_img_path)
    
    
    images_set2.append(image_left)
    steering_set2.append(steering_angle + 0.008 * steering_angle) 
    images_set2.append(image_right)
    steering_set2.append(steering_angle - 0.008 * steering_angle) 

images += images_set2
steering += steering_set2

lines = []
with open(data_file_paths[1] + '/driving_log.csv') as f:
    reader = csv.reader(f)
    for line in reader:
        lineout = [x.replace('/Users/wwymak/simulator_files2', '').replace(' ', '') for x in line]
        lines.append(lineout)

images_set3 = []
steering_set3 = []
for line in lines[1:]:
    img_path = line[0]
    image = cv2.imread(data_file_paths[1] + img_path)
    images_set3.append(image)
    steering_set3.append(line[3]) 

images += images_set3
steering += steering_set3


X_train = np.array(images)
y_train = np.array(steering)

In [37]:
# using resnet
# def resnet_custom():
#     inputs = Input(shape=X_train[0].shape)
#     cropped = Cropping2D(cropping=((70, 25), (0, 0)))(inputs)
#     processed = Lambda(lambda x: (x /255. - 0.5))(cropped) 
#     resized = Lambda(lambda x: K.tf.image.resize_images(x, (224, 224)))(processed)
#     res_model = keras.applications.resnet50.ResNet50(include_top=False, weights=None, input_tensor=resized, pooling=None,  input_shape = (224,224, 3))
#     predictions = Dense(1, name='final')(res_model.output)
#     model = Model(inputs=inputs, outputs=predictions)
#     model.compile(optimizer='adam',
#               loss='mse')
    
#     return model

In [38]:
K.clear_session()
batch_size = 32
epochs = 15
my_resnet = resnet_custom()

my_resnet_hist = my_resnet.fit(X_train, y_train, batch_size=batch_size,
                    epochs=epochs,validation_split=0.2,
                      verbose=0,shuffle=True, 
                    callbacks=[logging_callback, ModelCheckpoint('./models/my_resnet_sampledata_with_recovery.h5', save_best_only=True),
                              TensorBoard(log_dir='./logs' )]
                   )

ValueError: Error when checking target: expected final to have 4 dimensions, but got array with shape (35463, 1)

In [24]:
X_train[0].shape

(160, 320, 3)

In [8]:
# from https://github.com/keras-team/keras/blob/bc285462ad8ec9b8bc00bd6e09f9bcd9ae3d84a2/examples/cifar10_resnet.py
"""
ResNet v1
[a] Deep Residual Learning for Image Recognition
https://arxiv.org/pdf/1512.03385.pdf
ResNet v2
[b] Identity Mappings in Deep Residual Networks
https://arxiv.org/pdf/1603.05027.pdf
"""

from __future__ import print_function
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
import numpy as np
import os

# Training parameters
batch_size = 32
epochs = 200
data_augmentation = True
num_classes = 10

# Subtracting pixel mean improves accuracy
subtract_pixel_mean = True

# Model parameter
# ----------------------------------------------------------------------------
#           |      | 200-epoch | Orig Paper| 200-epoch | Orig Paper| sec/epoch
# Model     |  n   | ResNet v1 | ResNet v1 | ResNet v2 | ResNet v2 | GTX1090Ti
#           |      | %Accuracy | %Accuracy | %Accuracy | %Accuracy | v1 (v2)
# ----------------------------------------------------------------------------
# ResNet20  |  3   | 92.16     | 91.25     | -----     | NA        | 35
# ResNet32  |  5   | 92.46     | 92.49     | -----     | NA        | 50
# ResNet44  |  7   | 92.50     | 92.83     | -----     | NA        | 70
# ResNet56  |  9   | 92.71     | 93.03     | 92.60     | NA        | 90 (100)
# ResNet110 |  18  | 92.65     | 93.39     | 93.03     | 93.63     | 165(180)
# ---------------------------------------------------------------------------
n = 3

# Model version
# Orig paper: version = 1 (ResNet v1), Improved ResNet: version = 2 (ResNet v2)
version = 1

# Computed depth from supplied model parameter n
depth = n * 6 + 2

# Model name, depth and version
model_type = 'ResNet%dv%d' % (depth, version)


# # Input image dimensions.
# input_shape = x_train.shape[1:]

# # Normalize data.
# x_train = x_train.astype('float32') / 255
# x_test = x_test.astype('float32') / 255




def lr_schedule(epoch):
    """Learning Rate Schedule
    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.
    # Arguments
        epoch (int): The number of epochs
    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr


def resnet_block(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder
    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        conv_first (bool): conv-bn-activation (True) or
            activation-bn-conv (False)
    # Returns
        x (tensor): tensor as input to the next layer
    """
    if conv_first:
        x = Conv2D(num_filters,
                   kernel_size=kernel_size,
                   strides=strides,
                   padding='same',
                   kernel_initializer='he_normal',
                   kernel_regularizer=l2(1e-4))(inputs)
        x = BatchNormalization()(x)
        if activation:
            x = Activation(activation)(x)
        return x
    x = BatchNormalization()(inputs)
    if activation:
        x = Activation('relu')(x)
    x = Conv2D(num_filters,
               kernel_size=kernel_size,
               strides=strides,
               padding='same',
               kernel_initializer='he_normal',
               kernel_regularizer=l2(1e-4))(x)
    return x


def resnet_v1(input_shape, depth, num_classes=10):
    """ResNet Version 1 Model builder [a]
    Stacks of 2 x (3 x 3) Conv2D-BN-ReLU
    Last ReLU is after the shortcut connection.
    The number of filters doubles when the feature maps size
    is halved.
    The Number of parameters is approx the same as Table 6 of [a]:
    ResNet20 0.27M
    ResNet32 0.46M
    ResNet44 0.66M
    ResNet56 0.85M
    ResNet110 1.7M
    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)
    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])')
    # Start model definition.
    inputs = Input(shape=input_shape)
    num_filters = 16
    num_sub_blocks = int((depth - 2) / 6)

    x = resnet_block(inputs=inputs)
    # Instantiate convolutional base (stack of blocks).
    for i in range(3):
        for j in range(num_sub_blocks):
            strides = 1
            is_first_layer_but_not_first_block = j == 0 and i > 0
            if is_first_layer_but_not_first_block:
                strides = 2
            y = resnet_block(inputs=x,
                             num_filters=num_filters,
                             strides=strides)
            y = resnet_block(inputs=y,
                             num_filters=num_filters,
                             activation=None)
            if is_first_layer_but_not_first_block:
                x = resnet_block(inputs=x,
                                 num_filters=num_filters,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None)
            x = keras.layers.add([x, y])
            x = Activation('relu')(x)
        num_filters = 2 * num_filters

    # Add classifier on top.
    # v1 does not use BN after last shortcut connection-ReLU
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model


def resnet_v2(input_shape, depth, num_classes=10):
    """ResNet Version 2 Model builder [b]
    Stacks of (1 x 1)-(3 x 3)-(1 x 1) BN-ReLU-Conv2D or also known as
    bottleneck layer
    First shortcut connection per layer is 1 x 1 Conv2D.
    Second and onwards shortcut connection is identity.
    Features maps sizes: 16(input), 64(1st sub_block), 128(2nd), 256(3rd)
    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)
    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 56 or 110 in [b])')
    # Start model definition.
    inputs = Input(shape=input_shape)
    num_filters_in = 16
    num_filters_out = 64
    filter_multiplier = 4
    num_sub_blocks = int((depth - 2) / 9)

    # v2 performs Conv2D on input w/o BN-ReLU
    x = Conv2D(num_filters_in,
               kernel_size=3,
               padding='same',
               kernel_initializer='he_normal',
               kernel_regularizer=l2(1e-4))(inputs)

    # Instantiate convolutional base (stack of blocks).
    for i in range(3):
        if i > 0:
            filter_multiplier = 2
        num_filters_out = num_filters_in * filter_multiplier

        for j in range(num_sub_blocks):
            strides = 1
            is_first_layer_but_not_first_block = j == 0 and i > 0
            if is_first_layer_but_not_first_block:
                strides = 2
            y = resnet_block(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             conv_first=False)
            y = resnet_block(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_block(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if j == 0:
                x = Conv2D(num_filters_out,
                           kernel_size=1,
                           strides=strides,
                           padding='same',
                           kernel_initializer='he_normal',
                           kernel_regularizer=l2(1e-4))(x)
            x = keras.layers.add([x, y])

        num_filters_in = num_filters_out

    # Add classifier on top.
    # v2 has BN-ReLU before Pooling
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model

def resnet_custom(input_shape, depth, num_classes=1):
    inputs = Input(shape=input_shape)
    cropped = Cropping2D(cropping=((70, 25), (0, 0)))(inputs)
    processed = Lambda(lambda x: (x /255. - 0.5))(cropped) 
    resized = Lambda(lambda x: K.tf.image.resize_images(x, (224, 224)))(processed)
    
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])')
    # Start model definition.
    num_filters = 16
    num_sub_blocks = int((depth - 2) / 6)

    x = resnet_block(inputs=resized)
    # Instantiate convolutional base (stack of blocks).
    for i in range(3):
        for j in range(num_sub_blocks):
            strides = 1
            is_first_layer_but_not_first_block = j == 0 and i > 0
            if is_first_layer_but_not_first_block:
                strides = 2
            y = resnet_block(inputs=x,
                             num_filters=num_filters,
                             strides=strides)
            y = resnet_block(inputs=y,
                             num_filters=num_filters,
                             activation=None)
            if is_first_layer_but_not_first_block:
                x = resnet_block(inputs=x,
                                 num_filters=num_filters,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None)
            x = keras.layers.add([x, y])
            x = Activation('relu')(x)
        num_filters = 2 * num_filters

    # Add classifier on top.
    # v1 does not use BN after last shortcut connection-ReLU
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model


# Prepare callbacks for model saving and for learning rate adjustment.
# checkpoint = ModelCheckpoint(filepath=filepath,
#                              monitor='val_acc',
#                              verbose=1,
#                              save_best_only=True)

# lr_scheduler = LearningRateScheduler(lr_schedule)

# lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
#                                cooldown=0,
#                                patience=5,
#                                min_lr=0.5e-6)

# callbacks = [checkpoint, lr_reducer, lr_scheduler]



In [ ]:
K.clear_session()
batch_size = 16
epochs = 15

model = resnet_custom(input_shape=X_train[0].shape, depth=depth)

model.compile(loss='mse', optimizer='adam')


my_resnet_hist= model.fit(X_train, y_train, batch_size=batch_size,
                    epochs=epochs,validation_split=0.2,
                      verbose=1,shuffle=True, 
                    callbacks=[logging_callback, ModelCheckpoint('./models/resnet_test1_sampledata.h5', save_best_only=True),
                              TensorBoard(log_dir='./logs/resnet_test', histogram_freq=0, batch_size=32, )]
                   )


Train on 28370 samples, validate on 7093 samples
Epoch 1/15
28370/28370 [==============================] - 1233s 43ms/step - loss: 0.1669 - val_loss: 0.1917
Epoch 2/15
28370/28370 [==============================] - 1229s 43ms/step - loss: 0.1058 - val_loss: 0.1304